In [ ]:
# image segmentation

In [7]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage

gray = cv2.imread("giraffe.png", 0)
img = cv2.imread("giraffe.png")
blue, green, red = cv2.split(img)

In [2]:
def image_histogram_equalization(image, number_bins=256):   
  # get image histogram
  image_histogram, bins = np.histogram(image.flatten(), number_bins, density=True)
  # cumulative distribution function
  cdf = image_histogram.cumsum()  
  # normalize
  cdf = 255 * cdf / cdf[-1]
  # use linear interpolation of cdf to find new pixel values
  image_equalized = np.interp(image.flatten(), bins[:-1], cdf).astype('uint8')
  image_equalized = image_equalized.reshape((image.shape))
  return image_equalized

In [3]:
# separate image into binary using mean as threshold
def simple_segment(gray_img):
  new_gray = np.empty(gray_img.shape)
  for i in range(0, gray_img.shape[0]):
    for j in range(0, gray_img.shape[1]):
      if gray_img[i][j] > gray_img.mean():
          new_gray[i][j] = 255
      else:
          new_gray[i][j] = 0
  return new_gray

In [13]:
gray_seg = simple_segment(gray)
blue_seg = simple_segment(blue)
red_seg = simple_segment(red)
green_seg = simple_segment(green)

colour_seg = cv2.merge([blue_seg, green_seg, red_seg]) 

cv2.imwrite("giraffe_gray_seg.png", gray_seg)
cv2.imwrite("giraffe_colour_seg.png", colour_seg)

True

In [12]:
hist_gray = image_histogram_equalization(gray)
hist_blue = image_histogram_equalization(blue)
hist_green = image_histogram_equalization(green)
hist_red = image_histogram_equalization(red)

hist_gray = simple_segment(hist_gray)
hist_blue = simple_segment(hist_blue)
hist_green = simple_segment(hist_green)
hist_red = simple_segment(hist_red)

hist_seg = cv2.merge([hist_blue, hist_green, hist_red])

cv2.imwrite("giraffe_gray_hist_seg.png", hist_gray)
cv2.imwrite("giraffe_colour_hist_seg.png", hist_seg)

True

In [11]:
# otsu method
ret, thr = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)

hist_gray = image_histogram_equalization(gray)
ret1, thr1 = cv2.threshold(hist_gray, 0, 255, cv2.THRESH_OTSU)

cv2.imwrite("giraffe_otsu.png", thr)
cv2.imwrite("giraffe_hist_otsu.png", thr1)

True

In [ ]:
# image edge detection

In [23]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy import ndimage

gray = cv2.imread("grain.jpg", 0)
img = cv2.imread("giraffe.png")
blue, green, red = cv2.split(img)

In [16]:
# helper function to convert from PIL image format to OpenCV

def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2: 
        pass
    elif new_image.shape[2] == 3:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

In [24]:
# define Sobel filters
sobel_y = np.array([np.array([1, 2, 1]), np.array([0, 0, 0]), np.array([-1, -2, -1])])
sobel_x = np.array([np.array([-1, 0, 1]), np.array([-2, 0, 2]), np.array([-1, 0, 1])])

# remove noise
deblur = cv2.GaussianBlur(gray, (3, 3), 0)

# convolve filters with image
out_y = ndimage.convolve(deblur, sobel_y)
out_x = ndimage.convolve(deblur, sobel_x)
cv2.imwrite("grain_sobel_y.jpg", out_y)
cv2.imwrite("grain_sobel_x.jpg", out_x)

True

In [25]:
gradient_magnitude = np.sqrt(np.square(out_y) + np.square(out_x))
gradient_magnitude *= 255.0 / gradient_magnitude.max()
cv2.imwrite("grain_sobel.jpg", pil2cv(gradient_magnitude))

True

In [26]:
import cv2
import numpy as np

# remove noise
deblur = cv2.GaussianBlur(gray, (3, 3), 0)

# convolute with proper kernels
sobel_x_cv = cv2.Sobel(deblur, cv2.CV_64F, 1, 0, ksize=5)
sobel_y_cv = cv2.Sobel(deblur, cv2.CV_64F, 0, 1, ksize=5)
cv2.imwrite("grain_sobel_y_cv.jpg", out_y)
cv2.imwrite("grain_sobel_x_cv.jpg", out_x)

True

In [27]:
gradient_magnitude = np.sqrt(np.square(sobel_x_cv) + np.square(sobel_y_cv))
gradient_magnitude *= 255.0 / gradient_magnitude.max()
cv2.imwrite("grain_sobel_cv.jpg", pil2cv(gradient_magnitude))

True

In [28]:
# Canny's criteria

def canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
	# return the edged image
	return edged

edge = canny(gray, 0.33)
cv2.imwrite("grain_canny.jpg", edge)

True

In [35]:
# histogram equalized images

gray = cv2.imread("grain.jpg", 0)

gray_hist = image_histogram_equalization(gray)
deblur = cv2.GaussianBlur(gray_hist, (3, 3), 0)

# convolute with proper kernels
sobel_x_cv = cv2.Sobel(deblur, cv2.CV_64F, 1, 0, ksize=5)
sobel_y_cv = cv2.Sobel(deblur, cv2.CV_64F, 0, 1, ksize=5)
gradient_magnitude = np.sqrt(np.square(sobel_x_cv) + np.square(sobel_y_cv))
gradient_magnitude *= 255.0 / gradient_magnitude.max()
cv2.imwrite("grain_hist_sobel.jpg", pil2cv(gradient_magnitude))

edge = canny(gray_hist, 0.33)
cv2.imwrite("giraffe_hist_canny.png", edge)

True